In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.datasets.california_housing import fetch_california_housing

from sklearn.grid_search import GridSearchCV

## 1. 数据信息

In [3]:
housing = fetch_california_housing()

print("kes: ", housing.keys())
print("shape: ", housing['data'].shape)
# print("data detail: ", housing['data'][0])

kes:  dict_keys(['data', 'target', 'feature_names', 'DESCR'])
shape:  (20640, 8)


In [4]:
data_train, data_test, target_train, target_test = train_test_split(housing.data, housing.target, test_size=0.1, random_state=42)

## 2. 构建决策树

In [5]:
# 取部分特征构建树
dtr = tree.DecisionTreeRegressor(max_depth=2)
dtr.fit(data_train[:, [6, 7]], target_train)

dtr.score(data_test[:, [6, 7]], target_test)

0.14624466790651502

In [6]:
# 需下载安装：https://www2.graphviz.org/Packages/stable/windows/10/cmake/Release/x64/graphviz-install-2.44.1-win64.exe，添加到环境变量

dot_data = tree.export_graphviz(dtr,
                                out_file = None,
                                feature_names = housing.feature_names[6:8],
                                filled = True,
                                impurity = False,
                                rounded = True)

In [7]:
#  取所有特征构建树
dtr = tree.DecisionTreeRegressor(random_state=42)
dtr.fit(data_train, target_train)

dtr.score(data_test, target_test)

0.637318351331017

## 3. 随机森林

In [8]:
rfr = RandomForestRegressor( random_state = 42)
rfr.fit(data_train, target_train)

rfr.score(data_test, target_test)

0.7908649228096493

## 4. 自动挑参

#### 树模型参数:

-  1.criterion  gini  or  entropy

-  2.splitter  best or random 前者是在所有特征中找最好的切分点 后者是在部分特征中（数据量大的时候）

-  3.max_features  None（所有），log2，sqrt，N  特征小于50的时候一般使用所有的

-  4.max_depth  数据少或者特征少的时候可以不管这个值，如果模型样本量多，特征也多的情况下，可以尝试限制下

-  5.min_samples_split  如果某节点的样本数少于min_samples_split，则不会继续再尝试选择最优特征来进行划分如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

-  6.min_samples_leaf  这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝，如果样本量不大，不需要管这个值，大些如10W可是尝试下5

-  7.min_weight_fraction_leaf 这个值限制了叶子节点所有样本权重和的最小值，如果小于这个值，则会和兄弟节点一起被剪枝默认是0，就是不考虑权重问题。一般来说，如果我们有较多样本有缺失值，或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。

-  8.max_leaf_nodes 通过限制最大叶子节点数，可以防止过拟合，默认是"None”，即不限制最大的叶子节点数。如果加了限制，算法会建立在最大叶子节点数内最优的决策树。如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制具体的值可以通过交叉验证得到。

-  9.class_weight 指定样本各类别的的权重，主要是为了防止训练集某些类别的样本过多导致训练的决策树过于偏向这些类别。这里可以自己指定各个样本的权重如果使用“balanced”，则算法会自己计算权重，样本量少的类别所对应的样本权重会高。

- 10.min_impurity_split 这个值限制了决策树的增长，如果某节点的不纯度(基尼系数，信息增益，均方差，绝对差)小于这个阈值则该节点不再生成子节点。即为叶子节点 。
- n_estimators:要建立树的个数

In [9]:
tree_param_grid = { 'min_samples_split': list((3, 6, 9)), 'n_estimators': list((10, 50, 100))}

grid = GridSearchCV(RandomForestRegressor(), param_grid=tree_param_grid, cv=5)
grid.fit(data_train, target_train)
grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: 0.78450, std: 0.00453, params: {'min_samples_split': 3, 'n_estimators': 10},
  mean: 0.80441, std: 0.00422, params: {'min_samples_split': 3, 'n_estimators': 50},
  mean: 0.80719, std: 0.00335, params: {'min_samples_split': 3, 'n_estimators': 100},
  mean: 0.79044, std: 0.00442, params: {'min_samples_split': 6, 'n_estimators': 10},
  mean: 0.80501, std: 0.00411, params: {'min_samples_split': 6, 'n_estimators': 50},
  mean: 0.80679, std: 0.00435, params: {'min_samples_split': 6, 'n_estimators': 100},
  mean: 0.78863, std: 0.00825, params: {'min_samples_split': 9, 'n_estimators': 10},
  mean: 0.80465, std: 0.00417, params: {'min_samples_split': 9, 'n_estimators': 50},
  mean: 0.80649, std: 0.00503, params: {'min_samples_split': 9, 'n_estimators': 100}],
 {'min_samples_split': 3, 'n_estimators': 100},
 0.8071908828430434)

In [10]:
rfr = RandomForestRegressor(min_samples_split=3, n_estimators = 100, random_state = 42)

rfr.fit(data_train, target_train)
rfr.score(data_test, target_test)

0.8090829049653158